In [0]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import metrics
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [0]:
# Reading the data
train_data = pd.read_csv('/content/drive/My Drive/Deep learning & AI/CNN/fashion-mnist_train.csv')
test_data = pd.read_csv('/content/drive/My Drive/Deep learning & AI/CNN/fashion-mnist_test.csv')

In [45]:
train_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [46]:
test_data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,...,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,...,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,...,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,161,212,138,150,169,164,176,202,255,183,26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,129,221,255,255,215,...,255,255,254,252,255,101,196,254,252,100,0,0,0,0,0,168,232,225,0,41,172,147,148,153,155,146,137,141,143,137,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,204,235,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,69,0,2,0,0,0,0,0,0,0,0,0,0


In [0]:
# Initialising image shape
img_rows, img_cols = 28,28

In [0]:
# Seperating X and y columns of train dataset
X = np.array(train_data[train_data.columns.difference(['label'])])
y = np.array(train_data['label'])

#Test Data
test_X = np.array(test_data[test_data.columns.difference(['label'])])
test_y = np.array(test_data['label'])

In [0]:
# Splitting the training data into train and validation
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y =  train_test_split(X,y,test_size=0.2, random_state=1122)


In [0]:
#Convert the images into 3 dimensional arrays
if K.image_data_format() == 'channels_first':
    train_X = train_X.reshape(train_X.shape[0], 1, img_rows, img_cols)
    test_X = test_X.reshape(test_X.shape[0], 1, img_rows, img_cols)
    val_X = val_X.reshape(val_X.shape[0], 1, img_rows, img_cols)
    input_shape=(1, img_rows, img_cols)
else:
    train_X = train_X.reshape(train_X.shape[0], img_rows, img_cols, 1)
    test_X = test_X.reshape(test_X.shape[0], img_rows, img_cols, 1)
    val_X = val_X.reshape(val_X.shape[0], img_rows, img_cols, 1)
    input_shape=(img_rows, img_cols, 1)

In [51]:
val_X.shape

(12000, 28, 28, 1)

In [0]:
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
val_X = val_X.astype('float32')

In [0]:
train_X /= 255
test_X /= 255
val_X /= 255

In [54]:
print('train_X shape:', train_X.shape)
print(train_X.shape[0], 'train samples')
print(test_X.shape[0], 'test samples')

train_X shape: (48000, 28, 28, 1)
48000 train samples
10000 test samples


In [55]:
np.unique(val_y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [0]:
# convert class vectors to binary class matrices For target variables since it is multinomial classification
train_y = keras.utils.to_categorical(train_y, 10)
test_y = keras.utils.to_categorical(test_y, 10)
val_y = keras.utils.to_categorical(val_y, 10)

In [0]:
# Model initialization & creating network structure
model = Sequential()                                                                 # Model initialization 
model.add(Conv2D(32,(3,3),activation='relu',input_shape=input_shape)) # First convolutional layer
model.add(MaxPooling2D(pool_size=(2,2)))                                             # Max pool layer
model.add(Dropout(0.25))                                                             # Drop out layer
model.add(Conv2D(64, (3,3), activation='relu'))                         # Second convolutional layer
model.add(MaxPooling2D(pool_size=(2,2)))                                             # Max pool layer
model.add(Dropout(0.30))                                                             # Dopout layer 
model.add(Flatten())                                                                 # Flatten layer  
model.add(Dense(128, activation='relu'))                                             # Fully connected layer     
model.add(Dropout(0.4))                                                              # Dopout layer      
model.add(Dense(10, activation='softmax'))                                            # Output layer

In [58]:
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)             

In [0]:
#Model compilation
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [60]:
# Fit the  model on traiing data
#Training the model
model.fit(train_X, train_y,
          batch_size=256,
          epochs=50,
          verbose=1,
          validation_data=(val_X, val_y))

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 37s 761us/step - loss: 0.9941 - accuracy: 0.6262 - val_loss: 0.6338 - val_accuracy: 0.7506
Epoch 2/50
48000/48000 [==============================] - 36s 754us/step - loss: 0.6145 - accuracy: 0.7672 - val_loss: 0.4928 - val_accuracy: 0.8220
Epoch 3/50
48000/48000 [==============================] - 36s 756us/step - loss: 0.5307 - accuracy: 0.8034 - val_loss: 0.4620 - val_accuracy: 0.8294
Epoch 4/50
48000/48000 [==============================] - 36s 750us/step - loss: 0.4859 - accuracy: 0.8232 - val_loss: 0.4283 - val_accuracy: 0.8418
Epoch 5/50
48000/48000 [==============================] - 36s 751us/step - loss: 0.4554 - accuracy: 0.8328 - val_loss: 0.4213 - val_accuracy: 0.8413
Epoch 6/50
48000/48000 [==============================] - 36s 748us/step - loss: 0.4352 - accuracy: 0.8393 - val_loss: 0.3850 - val_accuracy: 0.8559
Epoch 7/50
48000/48000 [==============================] 

In [61]:
score=  model.evaluate(test_X,test_y)

10000/10000 [==============================] - 3s 262us/step


In [62]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.27256027460694315
Test accuracy: 0.8980000019073486


In [63]:
#  Test set predcition
model.predict(test_X)

array([[9.54402685e-01, 4.21761484e-07, 5.20459609e-04, ...,
        2.43942626e-08, 6.22750304e-05, 6.56053885e-07],
       [1.79697316e-11, 1.00000000e+00, 3.47459301e-12, ...,
        1.44956232e-15, 4.72927383e-12, 1.91346987e-15],
       [1.20772142e-03, 1.15554610e-06, 9.48304534e-01, ...,
        1.77995996e-09, 1.34804884e-06, 2.64583324e-08],
       ...,
       [6.66064963e-08, 1.16845197e-11, 1.29994338e-09, ...,
        5.95743721e-09, 9.99999881e-01, 5.17869359e-10],
       [1.76851526e-02, 7.64519646e-05, 1.42850622e-03, ...,
        1.18685675e-05, 9.72584665e-01, 3.15166544e-05],
       [5.19914813e-02, 6.83050573e-01, 9.32673365e-02, ...,
        4.25656908e-04, 6.05161907e-03, 4.84253076e-04]], dtype=float32)

In [0]:
# Test set classes predcition

test_pred_classes= model.predict_classes(test_X)

In [67]:
test_pred_classes

array([0, 1, 2, ..., 8, 8, 1])

In [68]:
import sklearn.metrics as metrics

metrics.accuracy_score(np.argmax(test_y, axis=1), model.predict_classes(test_X))

0.898

In [69]:
print(metrics.classification_report(np.argmax(test_y, axis=1), model.predict_classes(test_X)))

              precision    recall  f1-score   support

           0       0.81      0.88      0.84      1000
           1       0.99      0.98      0.99      1000
           2       0.79      0.88      0.84      1000
           3       0.90      0.92      0.91      1000
           4       0.85      0.84      0.84      1000
           5       0.98      0.96      0.97      1000
           6       0.80      0.64      0.71      1000
           7       0.97      0.90      0.93      1000
           8       0.97      0.98      0.98      1000
           9       0.91      0.99      0.95      1000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000

